In [4]:
import tensorflow as tf
import os
import sys
import collections

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
a = os.path.join(module_path, "strands_qsr_lib\qsr_lib\src3")

sys.path.append(a)
from rl import action_learner, action_learner_search, value_estimator
from progress_learner import EventProgressEstimator
import config
import project
# Need to add this import to load class
from project import Project
from importlib import reload
p = project.Project.load("../slidearound_hopstep_1_multiscale.proj")
from rl import block_movement_env
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
import plotting

In [38]:
c = config.Config()
c.no_of_loops = 1
# c.constraint_sigma = 0
reload(block_movement_env)
# reload(value_estimator)
reload(action_learner_search)
tf.reset_default_graph()

global_step = tf.Variable(0, name="global_step", trainable=False)

sess =  tf.Session()

policy_est = action_learner_search.PolicyEstimator(c)

sess.run(tf.global_variables_initializer())

with tf.variable_scope("model") as scope:
    print('-------- Load progress model ---------')
    progress_estimator = EventProgressEstimator(is_training=False, name = p.name, config = c)  

# Print out all variables that would be restored
for variable in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='model'):
    print (variable.name)

saver = tf.train.Saver(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='model'))
saver.restore(sess, '../progress.mod')

action_ln = action_learner_search.ActionLearner_Search(c, p, progress_estimator, 
                               policy_est, session = sess)

def action_policy(config):
    def boundary_constraint(action):
        for i in range(3):
            if action[0] < config.playground_x[0]:
                return False
            if action[0] > config.playground_x[0] + config.playground_dim[0]:
                return False
        
        return True
    
    def q(state, policy_estimator, no_of_actions = 1, verbose = False, 
       session = None):
        return action_learner_search.random_action_constraint(state, policy_estimator,
                    no_of_actions, verbose, session, boundary_constraint)
    
    return q

action_ln.learn(action_policy(c), verbose = True)

-------- Load progress model ---------
self.inputs.shape = (?, 20, 200)  after linear layer
output.shape = (10, 200) after LSTM
self.output.shape = (10,) after linear
self._targets.shape = (?,) 
model/SlideAround/linear/weight:0
model/SlideAround/linear/bias:0
model/SlideAround/lstm/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel:0
model/SlideAround/lstm/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/bias:0
model/SlideAround/output_linear/weight:0
model/SlideAround/output_linear/bias:0
INFO:tensorflow:Restoring parameters from ../progress.mod
{0: [Cube : [ 0.4646263   0.32597759  1.37317888], Cube : [ 0.4646263   0.32597759  1.37317888]], 1: [Cube : [ 0.22881703  0.06421508  0.63298786], Cube : [ 0.22881703  0.06421508  0.63298786]]}
2
{0: [Cube : [-0.37224119 -0.00780143  0.98648601], Cube : [-0.37224119 -0.00780143  0.98648601]], 1: [Cube : [-0.52909019  0.84030135  0.19386991], Cube : [-0.52909019  0.84030135  0.19386991]]}
2
{0: [Cube : [ 0.34172108  0.33704707  0.55677461], Cube : [ 0

action_level = 0
exploration_index = 0
{0: [Cube : [ 0.4646263   0.32597759  1.37317888], Cube : [ 0.4646263   0.32597759  1.37317888]], 1: [Cube : [ 0.22881703  0.06421508  0.63298786], Cube : [ 0.22881703  0.06421508  0.63298786]], 2: [], 3: []}
2


IndexError: list index out of range

In [34]:
a = block_movement_env.BlockMovementEnv(c, p.speed, progress_estimator = progress_estimator)

{0: [Cube : [ 0.14964207 -0.79181668  0.02251055], Cube : [ 0.14964207 -0.79181668  0.02251055]], 1: [Cube : [-0.62636363 -0.06188094  0.10401447], Cube : [-0.62636363 -0.06188094  0.10401447]]}
2


In [35]:
b = a.clone()

In [36]:
b.get_observation_start()

{0: [Cube : [ 0.14964207 -0.79181668  0.02251055], Cube : [ 0.14964207 -0.79181668  0.02251055]], 1: [Cube : [-0.62636363 -0.06188094  0.10401447], Cube : [-0.62636363 -0.06188094  0.10401447]]}
2


array([ 0.14964207, -0.79181668,  0.02251055,  0.14964207, -0.79181668,
        0.02251055, -0.62636363, -0.06188094,  0.10401447, -0.62636363,
       -0.06188094,  0.10401447])